In [8]:
# Test script for vtkCollisionDetectionFilter using data from autoScoper

import vtk
import time
# noinspection PyUnresolvedReferences
import vtkmodules.vtkInteractionStyle
# noinspection PyUnresolvedReferences
import vtkmodules.vtkRenderingFreeType
# noinspection PyUnresolvedReferences
import vtkmodules.vtkRenderingOpenGL2
from vtkmodules.vtkCommonColor import vtkNamedColors
from vtkmodules.vtkCommonMath import vtkMatrix4x4
from vtkmodules.vtkCommonTransforms import vtkTransform
from vtkmodules.vtkFiltersModeling import vtkCollisionDetectionFilter
from vtkmodules.vtkFiltersGeneral import vtkTransformPolyDataFilter
from vtkmodules.vtkFiltersSources import vtkSphereSource
from vtkmodules.vtkRenderingCore import (
    vtkActor,
    vtkPolyDataMapper,
    vtkRenderWindow,
    vtkRenderWindowInteractor,
    vtkRenderer,
    vtkTextActor
)

def loadStl(fname):
    """Load the given STL file, and return a vtkPolyData object for it."""
    reader = vtk.vtkSTLReader()
    reader.SetFileName(fname)
    reader.Update()
    polydata = reader.GetOutput()
    return polydata

# Function for testing collision between 2 polyData objects
def testCollision(polyData0, polyData1, contactMode):
    """Takes in 2 polyData objects and returns true if colliding, false otherwise"""
    collide = vtkCollisionDetectionFilter()
    collide.SetBoxTolerance(0.0)
    collide.SetCellTolerance(0.0)
    collide.SetNumberOfCellsPerNode(2)
    if contactMode == 0:
        collide.SetCollisionModeToAllContacts()
    elif contactMode == 1:
        collide.SetCollisionModeToFirstContact()
    else:
        collide.SetCollisionModeToHalfContacts()
    collide.GenerateScalarsOn()
    
    # Create transform required for filter
    # WARNING: Translate should be set to (0,0,0). This is
    # shifted since the data did not have colliding meshes
    matrix1 = vtkMatrix4x4()
    transform0 = vtkTransform()
    transform0.Translate(10.0, 0.0, 0.0)
    
    collide.SetInputData(0, polyData0) 
    collide.SetTransform(0, transform0)
    collide.SetInputData(1, polyData1) 
    collide.SetMatrix(1, matrix1)
    collide.Update()
    
    if collide.GetNumberOfContacts() > 0:
        return True
    return False

def main():
    
    # Contact options:
    # All contacts (0) finds all the contacting cell pairs with two points per collision.
    # First contact (1) quickly find the first contact point.
    # Half contacts (2) finds all the contacting cell pairs with one points per collision.
    #
    # For informing the particle swarm method, mode (1) will be the most performant
    contactMode = 1
    
    # If you want to see the collision, set to true, otherwise false
    visualize = False
    
    if visualize:
        collide = vtkCollisionDetectionFilter()
        collide.SetBoxTolerance(0.0)
        collide.SetCellTolerance(0.0)
        collide.SetNumberOfCellsPerNode(2)
        if contactMode == 0:
            collide.SetCollisionModeToAllContacts()
        elif contactMode == 1:
            collide.SetCollisionModeToFirstContact()
        else:
            collide.SetCollisionModeToHalfContacts()
        collide.GenerateScalarsOn()

        renderer = vtkRenderer()
        renderer.UseHiddenLineRemovalOn()

        renderWindow = vtkRenderWindow()
        renderWindow.SetSize(640, 480)

        numFiles = 10
        for i in range(0, numFiles):

            # Walk through the data as stl files, load them as vtkPolyData
            mc3 = "stl_col/heavy/mc3/"+str(i)+".stl"
            bone0 = loadStl(mc3)

            rad = "stl_col/heavy/rad/"+str(i)+".stl"
            bone1 = loadStl(rad)


            # Create transform to force collision between the bones
            # WARNING: Translate should be set to (0,0,0). This is
            # shifted since the data did not have colliding meshes
            matrix1 = vtkMatrix4x4()
            transform0 = vtkTransform()
            transform0.Translate(10.0, 0.0, 0.0)

            # Set data for the vtkCollisionDetectionFilter
            collide.SetInputData(0, bone0) 
            collide.SetTransform(0, transform0)
            collide.SetInputData(1, bone1) 
            collide.SetMatrix(1, matrix1)


            # Set up mappers and actors for visualization bones and
            # collision, this is unnecessary for collision, but nice 
            # for debugging
            colors = vtkNamedColors()

            mapper1 = vtkPolyDataMapper()
            mapper1.SetInputConnection(collide.GetOutputPort(0))
            mapper1.ScalarVisibilityOff()
            actor1 = vtkActor()
            actor1.SetMapper(mapper1)
            actor1.GetProperty().BackfaceCullingOn()
            actor1.SetUserTransform(transform0)
            actor1.GetProperty().SetDiffuseColor(colors.GetColor3d("Tomato"))
            actor1.GetProperty().SetRepresentationToWireframe()

            mapper2 = vtkPolyDataMapper()
            mapper2.SetInputConnection(collide.GetOutputPort(1))

            actor2 = vtkActor()
            actor2.SetMapper(mapper2)
            actor2.GetProperty().BackfaceCullingOn()
            actor2.SetUserMatrix(matrix1)

            mapper3 = vtkPolyDataMapper()
            mapper3.SetInputConnection(collide.GetContactsOutputPort())
            mapper3.SetResolveCoincidentTopologyToPolygonOffset()

            actor3 = vtkActor()
            actor3.SetMapper(mapper3)
            actor3.GetProperty().SetColor(colors.GetColor3d("Black"))
            actor3.GetProperty().SetLineWidth(3.0)

            txt = vtkTextActor()
            txt.GetTextProperty().SetFontSize(18)

            renderer.AddActor(actor1)
            renderer.AddActor(actor2)
            renderer.AddActor(actor3)
            renderer.AddActor(txt)
            renderer.SetBackground(colors.GetColor3d("Gray"))
            renderer.UseHiddenLineRemovalOn()
            renderWindow.AddRenderer(renderer)
            renderWindow.Render()
            renderer.GetActiveCamera().Azimuth(-90)

            cam = renderer.GetActiveCamera()

            renderWindow.SetWindowName('CollisionDetection')
            renderWindow.Render()

            if collide.GetNumberOfContacts() > 0:

                text = "Collision occurred on frame "+str(i)
                txt.SetInput(text)
                renderWindow.Render()
                time.sleep(1.0)

            time.sleep(1.0)

            renderWindow.Render()
            renderer.RemoveAllViewProps()
            renderer.GetActiveCamera().Azimuth(90)
            renderWindow.Render()

    if visualize == False:
        
        numFiles = 10
        for i in range(0, numFiles):

            # Walk through the data as stl files, load them as vtkPolyData
            mc3 = "stl_col/heavy/mc3/"+str(i)+".stl"
            bone0 = loadStl(mc3)
            
            rad = "stl_col/heavy/rad/"+str(i)+".stl"
            bone1 = loadStl(rad)
            if testCollision(bone0, bone1, contactMode):
                text = "Collision occurred on frame "+str(i)
                print(text)


if __name__ == '__main__':
    main()

Collision occurred on frame 4
Collision occurred on frame 5
Collision occurred on frame 6
Collision occurred on frame 7
Collision occurred on frame 8
